In [39]:
import numpy as np
import pandas as pd

In [40]:
df = pd.read_csv('./diabetes.csv')

In [41]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [42]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [43]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [44]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [45]:
X = scaler.fit_transform(X)

In [46]:
X.shape

(768, 8)

In [47]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [153]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Dropout

In [49]:
model = Sequential()
model.add(Dense(32, activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [50]:
model.fit(X_train,y_train,batch_size=32,epochs=10,validation_data = (X_test,y_test))

Epoch 1/10
20/20 [==============================] - 1s 21ms/step - loss: 0.7453 - accuracy: 0.5293 - val_loss: 0.6952 - val_accuracy: 0.5519
Epoch 2/10
20/20 [==============================] - 0s 5ms/step - loss: 0.6804 - accuracy: 0.6091 - val_loss: 0.6466 - val_accuracy: 0.5974
Epoch 3/10
20/20 [==============================] - 0s 4ms/step - loss: 0.6335 - accuracy: 0.6384 - val_loss: 0.6121 - val_accuracy: 0.6429
Epoch 4/10
20/20 [==============================] - 0s 5ms/step - loss: 0.5987 - accuracy: 0.6824 - val_loss: 0.5866 - val_accuracy: 0.6753
Epoch 5/10
20/20 [==============================] - 0s 4ms/step - loss: 0.5721 - accuracy: 0.6971 - val_loss: 0.5685 - val_accuracy: 0.7078
Epoch 6/10
20/20 [==============================] - 0s 4ms/step - loss: 0.5514 - accuracy: 0.7134 - val_loss: 0.5562 - val_accuracy: 0.6883
Epoch 7/10
20/20 [==============================] - 0s 4ms/step - loss: 0.5339 - accuracy: 0.7378 - val_loss: 0.5451 - val_accuracy: 0.7143
Epoch 8/10
20/20 [=

In [61]:
import keras_tuner as kt

In [113]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(32,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  optimizer=hp.Choice('optimizer',values=['Adam','sgd','rmsprop','adadelta'])
  model.compile(optimizer=optimizer, loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [115]:
tuner1 = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=10,directory='mydir',project_name='optimizers')

In [116]:
tuner1.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test),verbose=1)

Trial 4 Complete [00h 00m 02s]
val_accuracy: 0.5259740352630615

Best val_accuracy So Far: 0.7532467246055603
Total elapsed time: 00h 00m 08s


In [117]:
tuner1.get_best_hyperparameters()[0].values

{'optimizer': 'Adam'}

In [118]:
model1 = tuner1.get_best_models(num_models=1)[0]

In [119]:
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321 (1.25 KB)
Trainable params: 321 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [120]:
model1.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 [==============================] - 1s 14ms/step - loss: 0.5430 - accuracy: 0.7248 - val_loss: 0.5338 - val_accuracy: 0.7468
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5256 - accuracy: 0.7410 - val_loss: 0.5211 - val_accuracy: 0.7662
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5117 - accuracy: 0.7557 - val_loss: 0.5119 - val_accuracy: 0.7727
Epoch 10/100
20/20 [==============================] - 0s 5ms/step - loss: 0.5007 - accuracy: 0.7671 - val_loss: 0.5043 - val_accuracy: 0.7792
Epoch 11/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4909 - accuracy: 0.7752 - val_loss: 0.4978 - val_accuracy: 0.7857
Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4834 - accuracy: 0.7801 - val_loss: 0.4940 - val_accuracy: 0.7922
Epoch 13/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4765 - accuracy: 0.7866 - val_loss: 0.4913 - val_accuracy: 0.7792
Epoch 14

In [121]:
def build_model(hp):
  model = Sequential()

  units = hp.Int('units',8,32,step=8)
  model.add(Dense(units=units,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  optimizer=hp.Choice('optimizer',values=['Adam','sgd','rmsprop','adadelta'])
  model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [122]:
tuner2 = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=10,directory='mydir',project_name='nodes')

Reloading Tuner from mydir/nodes/tuner0.json


In [123]:
tuner2.search(X_train,y_train,epochs=10,validation_data=(X_test,y_test),verbose=True)

In [124]:
tuner2.get_best_hyperparameters()[0].values

{'units': 24, 'optimizer': 'rmsprop'}

In [125]:
model2 = tuner2.get_best_models(num_models=1)[0]

In [126]:
model2.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=11,validation_data=(X_test,y_test))

Epoch 12/100
20/20 [==============================] - 1s 13ms/step - loss: 0.5122 - accuracy: 0.7655 - val_loss: 0.5153 - val_accuracy: 0.8117
Epoch 13/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4976 - accuracy: 0.7655 - val_loss: 0.5070 - val_accuracy: 0.7987
Epoch 14/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4865 - accuracy: 0.7606 - val_loss: 0.5000 - val_accuracy: 0.8052
Epoch 15/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4772 - accuracy: 0.7671 - val_loss: 0.4961 - val_accuracy: 0.8052
Epoch 16/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4696 - accuracy: 0.7704 - val_loss: 0.4929 - val_accuracy: 0.8052
Epoch 17/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4646 - accuracy: 0.7687 - val_loss: 0.4915 - val_accuracy: 0.7987
Epoch 18/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4604 - accuracy: 0.7671 - val_loss: 0.4918 - val_accuracy: 0.7922
Epoch

In [130]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(72,activation='relu',input_dim=8))
  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
    units = hp.Int('units',8,32,step=8)
    model.add(Dense(units=units,activation='relu'))
  model.add(Dense(1,activation='sigmoid'))

  optimizer=hp.Choice('optimizer',values=['Adam','sgd','rmsprop','adadelta'])
  model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [131]:
tuner3 = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=10,directory='mydir',project_name='layers')

In [132]:
tuner3.search(X_train,y_train,epochs=10,validation_data=(X_test,y_test))

Trial 10 Complete [00h 00m 03s]
val_accuracy: 0.7662337422370911

Best val_accuracy So Far: 0.8051947951316833
Total elapsed time: 00h 00m 39s


In [133]:
tuner3.get_best_hyperparameters()[0].values

{'num_layers': 4, 'units': 24, 'optimizer': 'Adam'}

In [134]:
model3 = tuner3.get_best_models(num_models=1)[0]

In [135]:
model3.fit(X_train,y_train,epochs=100,initial_epoch=11,batch_size=32,validation_data=(X_test,y_test))

Epoch 12/100
20/20 [==============================] - 2s 14ms/step - loss: 0.4762 - accuracy: 0.7736 - val_loss: 0.4949 - val_accuracy: 0.7922
Epoch 13/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4520 - accuracy: 0.7948 - val_loss: 0.4993 - val_accuracy: 0.7922
Epoch 14/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4425 - accuracy: 0.7915 - val_loss: 0.5070 - val_accuracy: 0.7792
Epoch 15/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4322 - accuracy: 0.8046 - val_loss: 0.5217 - val_accuracy: 0.7922
Epoch 16/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4225 - accuracy: 0.8094 - val_loss: 0.5088 - val_accuracy: 0.7727
Epoch 17/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4185 - accuracy: 0.8078 - val_loss: 0.5179 - val_accuracy: 0.7727
Epoch 18/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4099 - accuracy: 0.8111 - val_loss: 0.5349 - val_accuracy: 0.7597
Epoch

In [146]:
def build_model(hp):
  model = Sequential()
  counter=0
  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
    if counter==0:
      model.add(
          Dense(
              hp.Int('units'+ str(i), min_value=8, max_value=128, step=8),
              activation = hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid','elu']),
              input_dim=8
          )
      )
    else:
      model.add(
          Dense(
              hp.Int('units'+ str(i), min_value=8, max_value=128, step=8),
              activation = hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid','elu'])
          )
      )
    counter=counter+1
  model.add(Dense(1,activation='sigmoid'))

  optimizer=hp.Choice('optimizer',values=['Adam','sgd','rmsprop','adadelta','nadam'])
  model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [147]:
tuner4 = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=10,directory='mydir',project_name='all')

In [148]:
tuner4.search(X_train,y_train,epochs=10,validation_data=(X_test,y_test))

Trial 10 Complete [00h 00m 04s]
val_accuracy: 0.7857142686843872

Best val_accuracy So Far: 0.7922077775001526
Total elapsed time: 00h 00m 49s


In [149]:
tuner4.get_best_hyperparameters()[0].values

{'num_layers': 3,
 'units0': 80,
 'activation0': 'sigmoid',
 'optimizer': 'nadam',
 'units1': 56,
 'activation1': 'tanh',
 'units2': 120,
 'activation2': 'elu',
 'units3': 8,
 'activation3': 'relu',
 'units4': 88,
 'activation4': 'sigmoid',
 'units5': 8,
 'activation5': 'elu',
 'units6': 128,
 'activation6': 'relu'}

In [151]:
model4 = tuner4.get_best_models(num_models=1)[0]

In [152]:
model4.fit(X_train,y_train,epochs=100,initial_epoch=11,batch_size=32,validation_data=(X_test,y_test))

Epoch 12/100
20/20 [==============================] - 2s 13ms/step - loss: 0.4748 - accuracy: 0.7785 - val_loss: 0.5094 - val_accuracy: 0.7403
Epoch 13/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4685 - accuracy: 0.7736 - val_loss: 0.5037 - val_accuracy: 0.7727
Epoch 14/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4718 - accuracy: 0.7704 - val_loss: 0.5098 - val_accuracy: 0.7403
Epoch 15/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4727 - accuracy: 0.7704 - val_loss: 0.5145 - val_accuracy: 0.7273
Epoch 16/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4694 - accuracy: 0.7687 - val_loss: 0.5108 - val_accuracy: 0.7273
Epoch 17/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4686 - accuracy: 0.7818 - val_loss: 0.5088 - val_accuracy: 0.7532
Epoch 18/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4663 - accuracy: 0.7818 - val_loss: 0.5124 - val_accuracy: 0.7273
Epoch

In [160]:
def build_model(hp):
  model = Sequential()
  counter=0
  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
    if counter==0:
      model.add(
          Dense(
              hp.Int('units'+ str(i), min_value=8, max_value=128, step=8),
              activation = hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid','elu']),
              input_dim=8
          )
      )
      model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    else:
      model.add(
          Dense(
              hp.Int('units'+ str(i), min_value=8, max_value=128, step=8),
              activation = hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid','elu'])
          )
      )
      model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    counter=counter+1
  model.add(Dense(1,activation='sigmoid'))

  optimizer=hp.Choice('optimizer',values=['Adam','sgd','rmsprop','adadelta','nadam'])
  model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [158]:
tuner5 = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=3,directory='mydir',project_name='final')

In [159]:
tuner5.search(X_train,y_train,epochs=10,validation_data=(X_test,y_test))

Trial 3 Complete [00h 00m 05s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.6428571343421936
Total elapsed time: 00h 00m 16s


In [161]:
tuner5.get_best_hyperparameters()[0].values

{'num_layers': 8,
 'units0': 112,
 'activation0': 'sigmoid',
 'dropout0': 0.4,
 'optimizer': 'nadam',
 'units1': 8,
 'activation1': 'relu',
 'dropout1': 0.1,
 'units2': 8,
 'activation2': 'relu',
 'dropout2': 0.1,
 'units3': 8,
 'activation3': 'relu',
 'dropout3': 0.1,
 'units4': 8,
 'activation4': 'relu',
 'dropout4': 0.1,
 'units5': 8,
 'activation5': 'relu',
 'dropout5': 0.1,
 'units6': 8,
 'activation6': 'relu',
 'dropout6': 0.1,
 'units7': 8,
 'activation7': 'relu',
 'dropout7': 0.1}

In [162]:
model5 = tuner5.get_best_models(num_models=1)[0]

In [163]:
model4.fit(X_train,y_train,epochs=100,initial_epoch=11,batch_size=32,validation_data=(X_test,y_test))

Epoch 12/100
20/20 [==============================] - 1s 12ms/step - loss: 0.4379 - accuracy: 0.7818 - val_loss: 0.5087 - val_accuracy: 0.7662
Epoch 13/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4354 - accuracy: 0.7899 - val_loss: 0.5153 - val_accuracy: 0.7792
Epoch 14/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4413 - accuracy: 0.7850 - val_loss: 0.5203 - val_accuracy: 0.7468
Epoch 15/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4327 - accuracy: 0.7883 - val_loss: 0.5110 - val_accuracy: 0.7597
Epoch 16/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4316 - accuracy: 0.7818 - val_loss: 0.5018 - val_accuracy: 0.7727
Epoch 17/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4337 - accuracy: 0.7866 - val_loss: 0.5126 - val_accuracy: 0.7662
Epoch 18/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4404 - accuracy: 0.7883 - val_loss: 0.5048 - val_accuracy: 0.7727
Epoch